## Encoder Performance

written by Isobel Mawby (i.mawby1@lancaster.ac.uk)

<div class="alert alert-block alert-info" style="font-size: 18px;">
    Imports
</div>

In [ ]:
import numpy as np
import torch  
from torch.utils.data import DataLoader
import sys

import Datasets
import TrainingMetrics

<div class="alert alert-block alert-info" style="font-size: 18px;">
    Set device
</div>

In [ ]:
device = 'cpu'

<div class="alert alert-block alert-info" style="font-size: 18px;">
    Config
</div>

In [ ]:
BATCH_SIZE = 64
TRAINING_FRACTION = 0.75

<div class="alert alert-block alert-info" style="font-size: 18px;">
    File 
</div>

In [ ]:
model_path = sys.path[0] + '/models/ContaminationClassifierModel_UVW'

<div class="alert alert-block alert-info" style="font-size: 18px;">
    Pull out things from file
</div>

In [ ]:
train_dataset, test_dataset = Datasets.get_classification_datasets(device, TRAINING_FRACTION)

print('Input(train):', train_dataset.input.shape)
print('Truth(train):', train_dataset.labels.shape)
print('')
print('Input(test):', test_dataset.input.shape)
print('Truth(test):', test_dataset.labels.shape)

In [ ]:
for class_index in TrainingMetrics.contamination_labels :
    print(f'{TrainingMetrics.contamination_strings[class_index]}: {torch.count_nonzero(train_dataset.labels == class_index).item()}')

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True, num_workers=0, generator=torch.Generator(device='cpu'))
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=False, num_workers=0, generator=torch.Generator(device='cpu'))

<div class="alert alert-block alert-info" style="font-size: 18px;">
    Get our predictions
</div>

In [ ]:
chosen_epoch = 0

model_path = f"{model_path}_{chosen_epoch}.pt"
model = torch.jit.load(model_path)

In [ ]:
pred_final_train = []
truth_train = []
pred_final_test = []
truth_test = []

model.eval()
    
with torch.no_grad():
    for x, label in train_dataloader:
        this_pred_final_train = torch.softmax(model(x), dim=1).detach().numpy() # Apply sigmoid at inference
        truth_train.extend(label.flatten().detach().numpy().tolist())
        pred_final_train.extend(this_pred_final_train.tolist())      

    for x, label in test_dataloader:
        this_pred_final_test = torch.softmax(model(x), dim=1).detach().numpy() # Apply sigmoid at inference  
        truth_test.extend(label.flatten().detach().numpy().tolist())
        pred_final_test.extend(this_pred_final_test.tolist())
        
# Turn into numpy arrays        
pred_final_train = np.array(pred_final_train)
truth_train = np.array(truth_train)
pred_final_test = np.array(pred_final_test)
truth_test = np.array(truth_test)

<div class="alert alert-block alert-info" style="font-size: 18px;">
    Make some post-training performance plots
</div>

In [ ]:
# For three classes: false track, true track, shower
TrainingMetrics.draw_confusion(pred_final_train, truth_train, [0.95, 0.95, 0.95], True) # Threshold is the same for all classes, additional 'reject' class

In [ ]:
TrainingMetrics.plot_scores(pred_final_train[:, 0], pred_final_test[:, 0], truth_train, truth_test, True)
TrainingMetrics.plot_scores(pred_final_train[:, 1], pred_final_test[:, 1], truth_train, truth_test, True)
TrainingMetrics.plot_scores(pred_final_train[:, 2], pred_final_test[:, 2], truth_train, truth_test, True)